In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [3]:
np.random.seed(500)

In [75]:
#To Start from Cell 91.
df=pd.read_csv("dataset.csv")

In [76]:
df.head()

,Unnamed: 0,url,repo,title,body,label
0,0,https://github.com/ZeroOne010101/PS2_Jaeger_Ac...,ZeroOne010101/PS2_Jaeger_Accountbot,Append new column if neccesary,The bot should append a new column to add a date.,bug
1,1,https://github.com/csufmse/student_information...,csufmse/student_information_system,student.currentsemester would show all section...,Added a filter for current semester\r\n\r\n@oc...,bug
2,2,https://github.com/sphinx-doc/sphinx/issues/9076,sphinx-doc/sphinx,Duplicate articles in TOC causes weird behavior,My team handles documentation and tutorials fo...,bug
3,3,https://github.com/ansible/ansible/pull/74218,ansible/ansible,fix typo in _collection_finder.py,\r\n\r\n##### SUMMARY\r\n<!--- Describe the ch...,bug
4,4,https://github.com/zimagi/zimagi/issues/54,zimagi/zimagi,`zimagi dbshell` does not find `psql`,"Moreover, it looks for it even if PostgreSQL h...",bug


In [36]:
df['body'] = [entry.lower() for entry in df['body']]
df['title'] = [entry.lower() for entry in df['title']]

df['body'].dropna(inplace=True)

df['body']= [word_tokenize(entry) for entry in df['body']]
df['title']= [word_tokenize(entry) for entry in df['title']]

In [67]:
df.head()

,title,body,label,body_final,title_final
0,"[append, new, column, if, neccesary]","[the, bot, should, append, a, new, column, to,...",3,"['bot', 'append', 'new', 'column', 'add', 'date']","['append', 'new', 'column', 'neccesary']"
1,"[student.currentsemester, would, show, all, se...","[added, a, filter, for, current, semester, @, ...",3,"['add', 'filter', 'current', 'semester', 'ocea...","['would', 'show', 'section', 'student', 'one']"
2,"[duplicate, articles, in, toc, causes, weird, ...","[my, team, handles, documentation, and, tutori...",3,"['team', 'handle', 'documentation', 'tutorial'...","['duplicate', 'article', 'toc', 'cause', 'weir..."
3,"[fix, typo, in, _collection_finder.py]","[#, #, #, #, #, summary, <, !, --, -, describe...",3,"['summary', 'describe', 'change', 'include', '...","['fix', 'typo']"
4,"[`, zimagi, dbshell, `, does, not, find, `, ps...","[moreover, ,, it, looks, for, it, even, if, po...",3,"['moreover', 'look', 'even', 'postgresql', 'tu...","['zimagi', 'dbshell', 'find', 'psql']"


In [ ]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index,entry in enumerate(df['body']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'body_final'] = str(Final_words)

for index,entry in enumerate(df['title']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'title_final'] = str(Final_words)


In [69]:
df.to_csv("svm-final.csv")


In [91]:
df1= pd.read_csv("svm-final.csv")
df1.head()

,Unnamed: 0,title,body,label,body_final,title_final
0,0,"['append', 'new', 'column', 'if', 'neccesary']","['the', 'bot', 'should', 'append', 'a', 'new',...",3,"['bot', 'append', 'new', 'column', 'add', 'date']","['append', 'new', 'column', 'neccesary']"
1,1,"['student.currentsemester', 'would', 'show', '...","['added', 'a', 'filter', 'for', 'current', 'se...",3,"['add', 'filter', 'current', 'semester', 'ocea...","['would', 'show', 'section', 'student', 'one']"
2,2,"['duplicate', 'articles', 'in', 'toc', 'causes...","['my', 'team', 'handles', 'documentation', 'an...",3,"['team', 'handle', 'documentation', 'tutorial'...","['duplicate', 'article', 'toc', 'cause', 'weir..."
3,3,"['fix', 'typo', 'in', '_collection_finder.py']","['#', '#', '#', '#', '#', 'summary', '<', '!',...",3,"['summary', 'describe', 'change', 'include', '...","['fix', 'typo']"
4,4,"['`', 'zimagi', 'dbshell', '`', 'does', 'not',...","['moreover', ',', 'it', 'looks', 'for', 'it', ...",3,"['moreover', 'look', 'even', 'postgresql', 'tu...","['zimagi', 'dbshell', 'find', 'psql']"


In [45]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\palak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\palak\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [92]:
X = df1['body_final'] + df1["title_final"]
y=df1['label']

In [93]:
X.shape

(22075,)

In [94]:
y.shape

(22075,)

In [95]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(X,y,test_size=0.2)

In [96]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [97]:
Tfidf_vect = TfidfVectorizer(max_features=10000)
Tfidf_vect.fit(X)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [102]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')

In [103]:
SVM.fit(Train_X_Tfidf,Train_Y)

SVC(gamma='auto', kernel='linear')

In [104]:
predictions_SVM = SVM.predict(Test_X_Tfidf)

In [105]:
print("SVM Accuracy Score ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score  66.22876557191393


In [106]:
from sklearn import metrics
print(metrics.classification_report(Test_Y, predictions_SVM))


              precision    recall  f1-score   support

           0       0.60      0.83      0.70      1479
           1       0.84      0.71      0.77       970
           2       0.57      0.46      0.51       581
           3       0.55      0.50      0.52       485
           4       0.75      0.54      0.63       509
           5       0.84      0.57      0.68       391

    accuracy                           0.66      4415
   macro avg       0.69      0.60      0.63      4415
weighted avg       0.68      0.66      0.66      4415

